# Lossy Compression With a beta-Variational Autoencoder

- This notebook is part of Problem Set 10, published on 28 June 2023 and discussed on 5 July 2023.
- You can download the full problem set (and solutions) from the [course website](https://robamler.github.io/teaching/compress23/).

In this problem, you will implement a toy beta-variational autoencoder ($\beta$-VAE), and you will then use it to lossily compress images.

## Beta-Variational Autoencoder ($\beta$-VAE)

### System Setup

Let's install and import some libraries, and set some global configurations.

In [ ]:
%pip install torch torchvision matplotlib tqdm

In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
import time
import sys

In [ ]:
USE_CUDA = False # change to True if you have a compatible GPU
RANDOM_SEED = 20230619
BATCH_SIZE = 100
EPOCHS = 10

torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if USE_CUDA else "cpu")

### Data Set: MNIST

In contrast to our setup for lossless compression, there is no point in binarizing the pixels for lossy compression.

In [ ]:
train_set = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

#### Visualize Some Training Data Points

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

def plot_samples(ax, loader, title):
    dat, _ = next(loader.__iter__())
    ax.set_title(title)
    ax.imshow(1 - make_grid(1 - dat.data[:64].float(), padding=2).permute((1, 2, 0)))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

plot_samples(ax1, train_loader, "training set")
plot_samples(ax2, test_loader, "test set")


### Baselines

Our baseline for lossy compression is the lossless compression that we implemented on the last problem set.
Recall that we obtained about 0.23 bits per pixel there.
Our lossy compression method should be able to achieve a lower bit rate without completely corrupting the images to a point where we can't recognize anything anymore.

### Definition of Model Components

The classes `EntropyBottleneck`, `EncoderModel`, `DecoderModel`, below implement the components of the $\beta$-VAE.

#### Problem 10.2 (a): `EntropyBottleneck`

The class `EntropyBottleneck` implements the bit rate term $\mathcal R(\theta,\phi,\mathbf x)$ from Eq. 3 on the problem set.
Here, we assume a prior $P_\theta(\mathbf z)=\mathcal N(0, S_\theta)$ where $S_\theta$ is a diagonal covariance matrix with learned entries.
More precisely, we learn a separate variance for each channel (last tensor dimension of `z`), but we assume that the prior is identical at each horizontal and vertical position (this will allow us again to apply the trained model to images of arbitrary size).

In [ ]:
class EntropyBottleneck(nn.Module):
    """The prior $P_\theta(Z)$."""

    def __init__(self, z_channels=3):
        super(EntropyBottleneck, self).__init__()
        self.prior_log_variance = torch.nn.Parameter(torch.ones((1, z_channels, 1, 1)))

    def forward(self, z):
        """Returns the term inside the expectation in Eq. 3 of the problem set, i.e.,
        $-\sum_i \log( \int_{z_i-0.5}^{z_i+0.5} p_i(z'_i) dz'_i )$ where p_i is a pdf
        of the i-th dimension of the prior. This allows us to estimate the rate term
        $KL( Q(Z|X=x) || P_\theta(Z) )$ for a box-shaped Q by sampling z from Q."""

        def antiderivative_of_normal(x, inverse_std):
            """Returns the antiderivative of a normal distribution with mean
            zero and standard deviation `1 / inverse_std`, evaluated at x."""
            return 0.5 * torch.erf((1 / np.sqrt(2)) * inverse_std * x)
        
        inverse_prior_std = torch.exp(-0.5 * self.prior_log_variance)
        
        # TODO: use the function `antiderivative_of_normal` defined above to evaluate
        # $\tilde p(z)$ as defined in Eq. 3 of the problem set (about 2 lines of code).

        return -p_tilde.log().sum()

    def prior_std(self):
        """Returns a tensor of prior standard deviations."""
        return torch.exp(0.5 * self.prior_log_variance)

    def entropy_per_latent_dim(self):
        """Returns $H_P(Z) / dim(Z)$ (mostly for debugging purpose)."""
        return 0.5 * torch.mean(np.log(2*np.pi) + 1 + self.prior_log_variance)

#### Problem 10.2 (b): `EncoderModel`

The `EncoderModel` is a neural network with weights $\phi$ that takes a batch of images $\mathbf x$ as input and outputs the parameters $\boldsymbol\mu_\phi(\boldsymbol x)$ of a box-shaped variational distribution that is centered at $\boldsymbol\mu_\phi(\boldsymbol x)$ and has width 1 in each coordinate direction.

In [ ]:
class EncoderModel(nn.Module):
    """A box-shaped variational distribution $Q(Z|X)$."""

    def __init__(self, hidden_channels=4, z_channels=3):
        super(EncoderModel, self).__init__()
        self.z_shape = [z_channels, 4, 4]
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=hidden_channels,
            kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=hidden_channels, out_channels=z_channels,
            kernel_size=5, stride=2)
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)

    def forward(self, x):
        """Returns the parameters that define $Q(Z|X=x)$ for given $x$."""
        hidden = torch.relu(self.conv1(x))
        q_mean = self.conv2(hidden)
        return q_mean
    
    def reparameterize(self, q_mean):
        """Draws $z ~ Q(Z|X=x)$ using the reparameterization trick (see Problem 8.2)."""
        # TODO: implement this method (1 or 2 lines of code);
        # Hints:
        # - the method `torch.rand_like` draws uniform samples from the interval [0, 1).
        # - for the VAE on the last problem set (with a Gaussian variational distribution),
        #   we had the following implementation:
        #   q_std = torch.exp(0.5 * q_log_variance) # Using torch.exp ensures that std > 0.
        #   eps = torch.randn_like(q_std) # `randn` is standard normal distribution.
        #   return q_mean + q_std * eps

#### Problem 10.2 (c): `DecoderModel`

The `DecoderModel` is a neural network with parameters $\theta$ that takes a batch of latent representations $\mathbf z$ as input and outputs a reconstructed image $\mathbf x'$.

In [ ]:
class DecoderModel(nn.Module):
    """Maps z to a reconstructed image."""

    def __init__(self, z_channels=3, hidden_channels=4):
        super(DecoderModel, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(
            in_channels=z_channels, out_channels=hidden_channels,
            kernel_size=5, stride=2, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(
            in_channels=hidden_channels, out_channels=1,
            kernel_size=5, stride=2, output_padding=1)
        nn.init.xavier_uniform_(self.deconv1.weight)
        nn.init.xavier_uniform_(self.deconv2.weight)

    def forward(self, z):
        """Returns a reconstructed image for given $z$."""
        hidden = torch.relu(self.deconv1(z))
        reconstruction = torch.clip(self.deconv2(hidden), 0, 1)
        return reconstruction

    def distortion(self, reconstruction, x):
        """Returns the 2-norm reconstruction error."""
        # TODO: return the distortion as defined below Eq. 1 on the problem
        # set, where $g(z)$ is given by the argument `reconstrution`.
        # (1 line of code)


### Problem 10.2 (d): Tying the Components Together

In [ ]:
def bit_rate_and_reconstruction(encoder_model, decoder_model, entropy_bottleneck, x):
    """Executes a round trip x --> z --> x';
    returns the estimated bit rate (to base 2), the distortion,
    the reconstructed images, and the total latent dimension."""
    q_mean = encoder_model(x)
    z = encoder_model.reparameterize(q_mean)
    bit_rate = entropy_bottleneck(z) / np.log(2)
    reconstructions = decoder_model(z)

    # TODO: in our VAE for lossless compression on Problem Set 9, we
    # had the following line:
    #
    # bit_rate_x_given_z = -decoder_model.log_likelihood(logits, x) / np.log(2)
    #
    # What should you replace this line with in order to train a VAE
    # for *lossy* data compression? Fill in 1 missing line of code.

    return bit_rate, distortion, reconstructions, z.numel()

### Training and Testing Functions

In [ ]:
def train_one_epoch(epoch, beta, train_loader, encoder_model, decoder_model,
                    entropy_bottleneck, optimizer, batches_between_logging=100):
    # Set all parts of the model into training mode.
    encoder_model.train()
    decoder_model.train()
    entropy_bottleneck.train()

    total_bit_rate = 0
    total_distortion = 0
    total_num_pixels = 0
    for batch_idx, (batch, _) in enumerate(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        bit_rate, distortion, _, num_z = bit_rate_and_reconstruction(
            encoder_model, decoder_model, entropy_bottleneck, batch)
        loss = beta * bit_rate + distortion
        loss.backward()
        optimizer.step()

        total_bit_rate += bit_rate
        total_distortion += distortion
        total_num_pixels += batch.numel()
        
        if batch_idx % batches_between_logging == 0:
            bpp = bit_rate / batch.numel()
            mse = distortion / batch.numel() # (MSE = mean squared error)
            loss_pp = loss / batch.numel() # (MSE = mean squared error)
            entropy_pz = entropy_bottleneck.entropy_per_latent_dim()
            entropy_pp = entropy_pz * num_z / batch.numel()
            print(
                f'Training epoch {epoch}, batch {batch_idx: 3d} of {len(train_loader)}: ' +
                f'BPP = {bpp:.4f}; MSE = {mse:.4f}; loss per pixel = {loss_pp:.4f};'
            )
            print(f'  prior entropy (per pixel, per latent dim): {entropy_pp:.4f}, {entropy_pz:.4f}')
            sys.stdout.flush()

    bpp = total_bit_rate / total_num_pixels
    mse = total_distortion / total_num_pixels
    return bpp, mse

In [ ]:
def test(test_loader, encoder_model, decoder_model, entropy_bottleneck):
    # Set all parts of the model into evaluation mode.
    encoder_model.eval()
    decoder_model.eval()
    entropy_bottleneck.eval()

    rendered_reconstructions = None
    total_bit_rate = 0
    total_distortion = 0
    total_num_pixels = 0

    with torch.no_grad():
        for i, (batch, _) in enumerate(test_loader):
            batch = batch.to(device)
            bit_rate, distortion, reconstructions, _ = bit_rate_and_reconstruction(
                encoder_model, decoder_model, entropy_bottleneck, batch)
            total_bit_rate += bit_rate
            total_distortion += distortion
            total_num_pixels += batch.numel()

        # Prepare a tensor that can be plotted to show a few reconstructions.
        comparison = torch.cat([batch[:8], reconstructions[:8]])
        rendered_reconstructions = make_grid(comparison.cpu()).permute((1, 2, 0))

        bpp = total_bit_rate / total_num_pixels
        mse = total_distortion / total_num_pixels

    return bpp, mse, rendered_reconstructions

### Train the VAE

In [ ]:
def training_loop(beta, train_loader, encoder_model, decoder_model, entropy_bottleneck, optimizer):
    # Create a placeholder figure that will be updated after each epoch below.
    figure, (samples_plot, reconstructions_plot) = plt.subplots(1, 2)
    samples_plot.set_title('samples')
    samples_plot.xaxis.set_visible(False)
    samples_plot.yaxis.set_visible(False)
    reconstructions_plot.set_title('test images and\ntheir reconstructions')
    reconstructions_plot.xaxis.set_visible(False)
    reconstructions_plot.yaxis.set_visible(False)
    hdisplay = display.display(figure, display_id=True)

    for epoch in range(1, EPOCHS + 1):
        start_time = time.time()
        train_bpp, train_mse = train_one_epoch(
            epoch, beta, train_loader, encoder_model, decoder_model, entropy_bottleneck,
            optimizer)
        test_bpp, test_mse, reconstructions = test(
            test_loader, encoder_model, decoder_model, entropy_bottleneck)
        end_time = time.time()
        train_loss = train_mse + beta * train_bpp
        test_loss = test_mse + beta * test_bpp
        print(
            f'Epoch {epoch} of {EPOCHS} ({end_time - start_time:.0f} sec): ' +
            f'training (BPP, MSE, loss): {train_bpp:.4f}, {train_mse:.4f}, {train_loss:.4f};  ' +
            f'test: {test_bpp:.4f}, {test_mse:.4f}, {test_loss:.4f}')
        sys.stdout.flush()

        # Plot some random samples $x ~ P(X)$ and reconstructions of test data.
        samples_plot.set_title(f'samples (epoch {epoch})')
        reconstructions_plot.set_title(
            f'test images and\ntheir reconstructions\n(epoch {epoch})')
        with torch.no_grad():
            samples_z = torch.randn([64] + encoder_model.z_shape).to(device)
            samples_x = decoder_model(samples_z).cpu()
            samples_plot.imshow(make_grid(samples_x).permute((1, 2, 0)))
        reconstructions_plot.imshow(reconstructions)
        hdisplay.update(figure)

    return train_bpp, train_mse, test_bpp, test_mse

In [ ]:
encoder_model = EncoderModel(z_channels=2).to(device)
decoder_model = DecoderModel(z_channels=2).to(device)
entropy_bottleneck = EntropyBottleneck(z_channels=2).to(device)

if USE_CUDA > 1:
    encoder_model = nn.DataParallel(encoder_model)
    decoder_model = nn.DataParallel(decoder_model)
    entropy_bottleneck = nn.DataParallel(entropy_bottleneck)

optimizer = optim.Adam(
    list(encoder_model.parameters())
    + list(decoder_model.parameters())
    + list(entropy_bottleneck.parameters()),
    lr=0.01)

training_loop(0.1, train_loader, encoder_model, decoder_model, entropy_bottleneck, optimizer)

### Problem 10.2 (e): Rate/Distortion Curves

**TODO:** implement a loop that trains $\beta$-VAEs for various values of $\beta$, and record the resulting bit rates and distortions after training.
Then plot the distortion as a function of the bit rate.
Come up with reasonable values for $\beta$ yourself.
Repeat the same for various dimensionalities of the latent space (argument `z_channels` in the three model constructors) and plot a rate/distortion curve for each tested value of `z_channels` in the same plot.

One typically finds that, for some dimensionalities of the latent space, the model performs better in the regime of low bit rates whereas for other dimensionalities, the model performs better in the regime of high bit rates.
For which values of `z_channel` do you expect the model to perform better in the high-bit-rate regime, and for which values do you expect it to perform better in the low-bit-rate regime?

## Lossy Data Compression With the Trained $\beta$-VAE

In this part, you'll use the $\beta$-VAE that you trained above to perform lossy data compression.
The bit rate and distortion of your compression method will depend on the value of $\beta$ that you set when you trained the VAE.

### System Setup

We'll again use the [`constriction`](https://bamler-lab.github.io/constriction/) library that we already used in Problems 7.3 and 9.2 to do the actual entropy coding.

In [ ]:
%pip install constriction~=0.3.1

In [ ]:
import constriction

In [ ]:
# Get a batch of test images that we can work with.
test_images, _ = next(test_loader.__iter__())

### Quantizing the Latent Space

At this point, quantization is actually easier than in the lossless case because we always quantize to integers (since the model was trained with a box-shaped variational distribution with fixed width 1).

In [ ]:
# NOTE: this cell needs to be rerun every time after training a new model.
prior_std = entropy_bottleneck.prior_std().detach().cpu().numpy()
Z_CUTOFF = 5 * int(round(prior_std.max() + 1.0))
quantized_gaussian = constriction.stream.model.QuantizedGaussian(-Z_CUTOFF, Z_CUTOFF)
print(f'prior standard deviations: {prior_std}')

In [ ]:
def quantize_z(z):
    """Map scaled latent representations to the nearest (integer) grid point."""
    return z.round().astype(np.int32)

def unquantize_z(quantized_z):
    """Convert an (integer) grid point to a (float) scaled latent representation."""
    return quantized_z.astype(np.float32)

### Encoder

In [ ]:
def encode_single_image(image, range_encoder):
    q_mean = encoder_model(image[None]) # [None] adds a batch dimension
    quantized_z = quantize_z(q_mean.detach().cpu().numpy())
    broadcasted_prior_std = np.broadcast_to(prior_std, quantized_z.shape)
    prior_mean = np.zeros_like(broadcasted_prior_std)
    range_encoder.encode(
        quantized_z.ravel(), quantized_gaussian,
        prior_mean.ravel(), broadcasted_prior_std.ravel())

    return q_mean.shape

### Problem 10.2 (f): Decoder

In [ ]:
def decode_single_image(range_decoder, z_shape):
    # TODO: implement the decoder: decode the quantized latent
    # representation, unquantize it, and then use the `decoder_model`
    # to obtain a reconstructed image, which you return.
    # (about 5 to 7 lines of code)

    return reconstruction[0]

### Testing the Compression Method

In [ ]:
def test_compression(images):
    """Encodes and decodes all images, verifies correctness, prints
    the overall number of bits per pixel (BPP), and returns 3 tensors:
    the BPPs for all images, the MSEs for all images, and the
    reconstructed images."""

    encoder = constriction.stream.queue.RangeEncoder()

    bpps = np.empty(len(images))
    for i, image in enumerate(images):
        num_bits_before = encoder.num_bits()
        z_shape = encode_single_image(image, encoder)
        num_bits_after = encoder.num_bits()
        bpps[i] = (num_bits_after - num_bits_before) / image.numel()

    print(f"Encoded {len(images)} images with a BPP of {bpps.mean():.4f} ± {bpps.std():.4f}")
    compressed = encoder.get_compressed()

    # Verify that the method `num_bits` didn't lie:
    assert compressed.dtype == np.uint32
    assert compressed.size * 32 == num_bits_after

    range_decoder = constriction.stream.queue.RangeDecoder(compressed)
    reconstructions = torch.stack([decode_single_image(range_decoder, z_shape) for _ in range(len(images))])
    mses = ((reconstructions - images)**2).mean(axis=(1, 2, 3))
    print(f"Decoded all images, observing a mean squared error of {mses.mean():.4f} ± {mses.std():.4f}")

    return bpps, mses, reconstructions

In [ ]:
bpps, mses, reconstructions = test_compression(test_images)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

def plot_samples(ax, imgs, title):
    ax.set_title(title)
    ax.imshow(make_grid(imgs[:64]).permute((1, 2, 0)))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

plot_samples(ax1, test_images, "test images")
plot_samples(ax2, reconstructions, "reconstructions")

## Generalization to Out of Distribution Data (OOD)

### Load The OOD Data Set

Make sure that you have the file `smallkkanji2.npz` that was included with last week's problem set, and that it is located at the right path.

In [ ]:
import torch.utils.data as data_utils

kanji_data = np.load('kkanji2/smallkkanji2.npz')['data'][:, None, :, :]
kanji_data = torch.tensor(kanji_data.astype(np.float32) / 255)
mock_kanji_labels = torch.zeros((len(kanji_data,))) # We don't need labels but pytorch is very opinionated.

kanji_set = data_utils.TensorDataset(kanji_data, mock_kanji_labels)
kanji_loader = torch.utils.data.DataLoader(kanji_set, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(1 - make_grid(1 - kanji_data[:64], padding=4).permute((1, 2, 0)))
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

### Try Encoder and Inspect Latent Dimensions

Even though the VAE was trained on images of a different dimension, it should also work on these images (NB: we are a bit lucky here that the padding works out; in general, deconvolutional layers with a $\mathtt{stride}\neq 1$ can reconstruct images whose dimensions differ slightly from the original, which has to be than resolved by output padding).

In [ ]:
train_batch, _ = next(train_loader.__iter__())
q_mean = encoder_model(train_batch)
logits = decoder_model(q_mean)
train_batch.shape, q_mean.shape, reconstructions.shape

In [ ]:
kanji_batch, _ = next(kanji_loader.__iter__())
q_mean = encoder_model(kanji_batch)
reconstructions = decoder_model(q_mean)
kanji_batch.shape, q_mean.shape, reconstructions.shape

### Test VAE on OOD Data Set

In [ ]:
kanji_bpp, kanji_mse, kanji_reconstruction_plot = test(kanji_loader, encoder_model, decoder_model, entropy_bottleneck)
print(f'OOD data has BPP = {kanji_bpp:.4f}; MSE = {kanji_mse:.4f}')
ax = plt.imshow(kanji_reconstruction_plot.detach());
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

### Test Compression Method on OOD Data Set

In [ ]:
test_compression(kanji_data);